In [ ]:
##This script was used for obtaining scores with the keras neural network using k-mers.
##The scores obtained are worse than with Pfam counts

In [234]:
cd C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\aminoAcidDictionaries

C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\aminoAcidDictionaries


In [235]:
import numpy as np

In [236]:
kmerToColumn={}
i=0
numberOfClusters=0
inputFile = open("length3Processed.txt")
for line in inputFile:
    if line [0:3]== "BGC":
        numberOfClusters +=1
        continue
    line = line.strip("\n")
    line = line.strip(";")
    line = line.split(";")
    for kmer in line:
        kmer=kmer.split(":")
        if kmer[0] not in kmerToColumn:
            kmerToColumn[kmer[0]]=i
            i+=1
inputFile.close()

In [237]:
inputVectorMatrix= np.zeros((numberOfClusters,i))
i=0
inputFile = open("length3Processed.txt")
for line in inputFile:
    if line [0:3]== "BGC":
        continue
    thisClusterDictionary={}
    line = line.strip("\n")
    line = line.strip(";")
    line = line.split(";")
    for kmer in line:
        kmer=kmer.split(":")
        thisClusterDictionary[kmer[0]] =kmer[1]
    for key in kmerToColumn:
        if key in thisClusterDictionary.keys():
            inputVectorMatrix[i,kmerToColumn[key]] = thisClusterDictionary[key]
    i+=1
inputFile.close()

In [238]:
def convertStringIntoVector (String):##Receives a binary vector in the form of a String and converts it into a numpy array
    intoList=list(String)
    vector=np.asarray(intoList,dtype=int)
    return vector

In [239]:
cd C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints

C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints


In [240]:
fingerprintFile=open ("fingerprintsOnlyWithImportantSubstructures.txt") ##Normal fingerprints can also be used
fingerprintList=[]##List of strings
for line in fingerprintFile:
    line=line.strip("\n")
    fingerprintList.append(line[11:])
fingerprintFile.close()

In [241]:
fingerprintVectorList=[]##List of vectors
numberOfClusters=0
for fingerprintString in fingerprintList:
    fingerprintVectorList.append(convertStringIntoVector(fingerprintString))
    numberOfClusters+=1
yData=fingerprintVectorList
xData=inputVectorMatrix

In [242]:
##THIS can be activated for normalisation of each column
#numberOfFeatures= xData.shape[1]
#for i in range (0,numberOfFeatures):
  #  mean=xData.mean(0)[i]
   # sd= xData.std(0)[i]
    #for j in range (0, numberOfClusters):
    #    xData[j][i]= (xData[j][i]-mean)/sd

In [243]:
yData=np.concatenate(yData,axis=0)
yData=yData.reshape(numberOfClusters,246)
#The reshape puts the data as one row for each training example
inputDimension= xData.shape[1]

In [244]:
import random
#Creates a list of Random Numbers as big as the number of Clusters we have
#Then we will select which ones we have for each subset
listOfRandomNumbers=random.sample(range(numberOfClusters), numberOfClusters)

In [245]:
xTestData = np.asarray([xData[i] for i in listOfRandomNumbers[618:824]],dtype=int)
yTestData = np.asarray([yData[i] for i in listOfRandomNumbers[618:824]],dtype=int)
xTrainingData = np.asarray([xData[i] for i in (listOfRandomNumbers[:618]+listOfRandomNumbers[824:])],dtype=int)
yTrainingData = np.asarray([yData[i]for i in (listOfRandomNumbers[:618]+listOfRandomNumbers[824:])],dtype=int)

In [246]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
from keras import initializers
nn=[inputDimension, 50, 246]
##The learning rate is changed after a number of epochs, the weights of the model are saved and loaded each time

In [248]:
lr=0.01
model=Sequential()
model.add(Dense(nn[0], input_dim=inputDimension, activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1) ))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[2], activation = 'sigmoid',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=lr), metrics = ['accuracy'])
model.fit(xTrainingData, yTrainingData, epochs=150)
model.save_weights('trial.h5')

Epoch 1/150
825/825 [==============================] - 18s 22ms/step - loss: 0.8239 - acc: 0.5237
Epoch 2/150
825/825 [==============================] - 14s 17ms/step - loss: 0.7370 - acc: 0.5417
Epoch 3/150
825/825 [==============================] - 14s 17ms/step - loss: 0.7118 - acc: 0.5550
Epoch 4/150
825/825 [==============================] - 15s 18ms/step - loss: 0.6980 - acc: 0.5683
Epoch 5/150
825/825 [==============================] - 14s 17ms/step - loss: 0.6878 - acc: 0.5810
Epoch 6/150
825/825 [==============================] - 14s 17ms/step - loss: 0.6789 - acc: 0.5940
Epoch 7/150
825/825 [==============================] - 14s 17ms/step - loss: 0.6704 - acc: 0.6072
Epoch 8/150
825/825 [==============================] - 15s 18ms/step - loss: 0.6615 - acc: 0.6208
Epoch 9/150
825/825 [==============================] - 15s 18ms/step - loss: 0.6515 - acc: 0.6357
Epoch 10/150
825/825 [==============================] - 15s 18ms/step - loss: 0.6400 - acc: 0.6511
Epoch 11/150
825/82

In [249]:
lr=0.003
model=Sequential()
model.add(Dense(nn[0], input_dim=inputDimension, activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1) ))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[2], activation = 'sigmoid',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=lr), metrics = ['accuracy'])
model.load_weights("trial.h5")
model.fit(xTrainingData, yTrainingData, epochs=150)
model.save_weights('trial.h5')

Epoch 1/150
825/825 [==============================] - 17s 21ms/step - loss: 0.1527 - acc: 0.9398
Epoch 2/150
825/825 [==============================] - 14s 17ms/step - loss: 0.1526 - acc: 0.9399
Epoch 3/150
825/825 [==============================] - 14s 17ms/step - loss: 0.1526 - acc: 0.9399
Epoch 4/150
825/825 [==============================] - 14s 17ms/step - loss: 0.1526 - acc: 0.9399
Epoch 5/150
825/825 [==============================] - 14s 17ms/step - loss: 0.1526 - acc: 0.9399
Epoch 6/150
825/825 [==============================] - 14s 17ms/step - loss: 0.1525 - acc: 0.9399
Epoch 7/150
825/825 [==============================] - 14s 17ms/step - loss: 0.1525 - acc: 0.9400
Epoch 8/150
825/825 [==============================] - 14s 17ms/step - loss: 0.1525 - acc: 0.9400
Epoch 9/150
825/825 [==============================] - 14s 17ms/step - loss: 0.1525 - acc: 0.9398
Epoch 10/150
825/825 [==============================] - 14s 17ms/step - loss: 0.1525 - acc: 0.9399
Epoch 11/150
825/82

In [221]:
lr=0.001
model=Sequential()
model.add(Dense(nn[0], input_dim=inputDimension, activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1) ))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[2], activation = 'sigmoid',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=lr), metrics = ['accuracy'])
model.load_weights("trial.h5")
model.fit(xTrainingData, yTrainingData, epochs=150)
model.save_weights('trial.h5')

Epoch 1/150
825/825 [==============================] - 1s 2ms/step - loss: 0.1547 - acc: 0.9396
Epoch 2/150
825/825 [==============================] - 0s 162us/step - loss: 0.1547 - acc: 0.9396
Epoch 3/150
825/825 [==============================] - 0s 158us/step - loss: 0.1547 - acc: 0.9396
Epoch 4/150
825/825 [==============================] - 0s 158us/step - loss: 0.1547 - acc: 0.9397
Epoch 5/150
825/825 [==============================] - 0s 170us/step - loss: 0.1547 - acc: 0.9396
Epoch 6/150
825/825 [==============================] - 0s 153us/step - loss: 0.1547 - acc: 0.9396
Epoch 7/150
825/825 [==============================] - 0s 161us/step - loss: 0.1547 - acc: 0.9396
Epoch 8/150
825/825 [==============================] - 0s 173us/step - loss: 0.1547 - acc: 0.9397
Epoch 9/150
825/825 [==============================] - 0s 156us/step - loss: 0.1547 - acc: 0.9396
Epoch 10/150
825/825 [==============================] - 0s 169us/step - loss: 0.1547 - acc: 0.9396
Epoch 11/150
825/825 

In [226]:
def gaussianKernel (vector1, vector2):##Does the gaussian Kernel for two vectors
    gamma=0.1
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [233]:
predictions = model.predict(xTestData)
for i in range (0, len (predictions)):
    for j in range (0, len (predictions[0])):
        if predictions[i][j]<0.5: ##Rounds up and down the predictions
            predictions[i][j] = 0
        else:
            predictions[i][j] = 1
score = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        score[i,j]= gaussianKernel(predictions[i],yTestData[j])
np.save("kmersKerasScore",score)##Creates the ouputfile ready to be opened with analysing Scores notebook

105
101
